In [1]:

import os
import copy
import time
import pickle
import numpy as np
import easydict
from tqdm import tqdm

import torch
from tensorboardX import SummaryWriter

from options import args_parser
from update import LocalUpdate, test_inference
from models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar, CNNCifar_fedVC, CNNCifar_VCBN, CNNCifar_VCGN, CNNCifar_WS
from utils import get_dataset, average_weights, exp_details, get_logger, CSVLogger
from fed_cifar100 import load_partition_data_federated_cifar100
from resnet_gn import resnet18
from resnet import ResNet32_test
from vgg import vgg11_bn, vgg11
import random
import logging
import datetime
from test import main_test


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

In [3]:
net_loss_list = []
net_acc_list = []
net_loss_itr_list = []
net_grad_conv_list = []
net_grad_fc_list = []
net_grad_norm_list = []

## 기본 lr 0.1

In [4]:
args = easydict.EasyDict({
    "model": 'resnet18',
    'dataset': 'cifar100',
    'gpu': 0,
    'iid': 2,
    'epochs': 4000,
    'optimizer': 'no_weight_decay',
    'seed': 0,
    'norm': 'group_norm',
    'num_users': 500,
    'frac': 0.02,
    'local_ep': 1,
    'local_bs': 20,
    'lr': 0.1,
    'momentum': 0.9,
    'kernel_num': 9,
    'kernel_sizes': '3,4,5',
    'num_channnels': '1',
    'num_filters': 32,
    'max_pool': 'True',
    'num_classes': 10,
    'unequal': 0,
    'stopping_rounds': 10,
    'verbose': 0,
    'hold_normalize': 0,
    'save_path': '../save/checkpoint',
    'exp_folder': 'FedCifar100_frac0.02',
    'resume': None,
    'lr_decay': 1
})

train_loss, val_acc_list, batch_loss_list, conv_grad_list, fc_grad_list, total_grad_list = main_test(args)
net_loss_list.append(train_loss)
net_acc_list.append(val_acc_list)
net_loss_itr_list.append(batch_loss_list)
net_grad_conv_list.append(conv_grad_list)
net_grad_fc_list.append(fc_grad_list)
net_grad_norm_list.append(total_grad_list)

INFO:root:{'model': 'resnet18', 'dataset': 'cifar100', 'gpu': 0, 'iid': 2, 'epochs': 4000, 'optimizer': 'no_weight_decay', 'seed': 0, 'norm': 'group_norm', 'num_users': 500, 'frac': 0.02, 'local_ep': 1, 'local_bs': 20, 'lr': 0.1, 'momentum': 0.9, 'kernel_num': 9, 'kernel_sizes': '3,4,5', 'num_channnels': '1', 'num_filters': 32, 'max_pool': 'True', 'num_classes': 10, 'unequal': 0, 'stopping_rounds': 10, 'verbose': 0, 'hold_normalize': 0, 'save_path': '../save/checkpoint\\FedCifar100_frac0.02', 'exp_folder': 'FedCifar100_frac0.02', 'resume': None, 'lr_decay': 1}
03/08 05:09:59 PM | {'model': 'resnet18', 'dataset': 'cifar100', 'gpu': 0, 'iid': 2, 'epochs': 4000, 'optimizer': 'no_weight_decay', 'seed': 0, 'norm': 'group_norm', 'num_users': 500, 'frac': 0.02, 'local_ep': 1, 'local_bs': 20, 'lr': 0.1, 'momentum': 0.9, 'kernel_num': 9, 'kernel_sizes': '3,4,5', 'num_channnels': '1', 'num_filters': 32, 'max_pool': 'True', 'num_classes': 10, 'unequal': 0, 'stopping_rounds': 10, 'verbose': 0, 'ho

num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256


  0%|                                                                                         | 0/4000 [00:00<?, ?it/s]INFO:root: 
Avg Training Stats after 1 global rounds:
03/08 05:10:36 PM |  
Avg Training Stats after 1 global rounds:
INFO:root:Training Loss : 4.30235499382019
03/08 05:10:36 PM | Training Loss : 4.30235499382019
INFO:root:Train Accuracy: 0.83% 

03/08 05:10:36 PM | Train Accuracy: 0.83% 

  0%|                                                                               | 1/4000 [00:03<4:16:31,  3.85s/it]INFO:root: 
Avg Training Stats after 2 global rounds:
03/08 05:10:38 PM |  
Avg Training Stats after 2 global rounds:
INFO:root:Training Loss : 4.571623449325562
03/08 05:10:38 PM | Training Loss : 4.571623449325562
INFO:root:Train Accuracy: 1.10% 

03/08 05:10:38 PM | Train Accuracy: 1.10% 

  0%|                                                                               | 2/4000 [00:05<3:06:15,  2.80s/it]INFO:root: 
Avg Training Stats after 3 global rounds:
03/

INFO:root:Train Accuracy: 2.24% 

03/08 05:11:14 PM | Train Accuracy: 2.24% 

  0%|▍                                                                             | 20/4000 [00:41<2:12:17,  1.99s/it]INFO:root: 
Avg Training Stats after 21 global rounds:
03/08 05:11:16 PM |  
Avg Training Stats after 21 global rounds:
INFO:root:Training Loss : 3.9898415660858157
03/08 05:11:16 PM | Training Loss : 3.9898415660858157
INFO:root:Train Accuracy: 1.56% 

03/08 05:11:16 PM | Train Accuracy: 1.56% 

  1%|▍                                                                             | 21/4000 [00:43<2:12:09,  1.99s/it]INFO:root: 
Avg Training Stats after 22 global rounds:
03/08 05:11:18 PM |  
Avg Training Stats after 22 global rounds:
INFO:root:Training Loss : 3.911823735237121
03/08 05:11:18 PM | Training Loss : 3.911823735237121
INFO:root:Train Accuracy: 1.41% 

03/08 05:11:18 PM | Train Accuracy: 1.41% 

  1%|▍                                                                             | 22/40

INFO:root:Training Loss : 3.8653843069076537
03/08 05:11:54 PM | Training Loss : 3.8653843069076537
INFO:root:Train Accuracy: 2.18% 

03/08 05:11:54 PM | Train Accuracy: 2.18% 

  1%|▊                                                                             | 40/4000 [01:21<2:11:47,  2.00s/it]INFO:root: 
Avg Training Stats after 41 global rounds:
03/08 05:11:56 PM |  
Avg Training Stats after 41 global rounds:
INFO:root:Training Loss : 4.058887205123902
03/08 05:11:56 PM | Training Loss : 4.058887205123902
INFO:root:Train Accuracy: 2.80% 

03/08 05:11:56 PM | Train Accuracy: 2.80% 

  1%|▊                                                                             | 41/4000 [01:23<2:11:58,  2.00s/it]INFO:root: 
Avg Training Stats after 42 global rounds:
03/08 05:11:58 PM |  
Avg Training Stats after 42 global rounds:
INFO:root:Training Loss : 3.643276863098145
03/08 05:11:58 PM | Training Loss : 3.643276863098145
INFO:root:Train Accuracy: 2.28% 

03/08 05:11:58 PM | Train Accuracy: 

...

 99%|███████████████████████████████████████████████████████████████████████████▎| 3961/4000 [3:57:17<03:42,  5.70s/it]INFO:root: 
Avg Training Stats after 3962 global rounds:
03/08 09:07:55 PM |  
Avg Training Stats after 3962 global rounds:
INFO:root:Training Loss : 4.858103158767335e-05
03/08 09:07:55 PM | Training Loss : 4.858103158767335e-05
INFO:root:Train Accuracy: 30.27% 

03/08 09:07:55 PM | Train Accuracy: 30.27% 

 99%|███████████████████████████████████████████████████████████████████████████▎| 3962/4000 [3:57:22<03:33,  5.63s/it]INFO:root: 
Avg Training Stats after 3963 global rounds:
03/08 09:08:00 PM |  
Avg Training Stats after 3963 global rounds:
INFO:root:Training Loss : 4.8799192372825926e-05
03/08 09:08:01 PM | Training Loss : 4.8799192372825926e-05
INFO:root:Train Accuracy: 30.27% 

03/08 09:08:01 PM | Train Accuracy: 30.27% 

 99%|███████████████████████████████████████████████████████████████████████████▎| 3963/4000 [3:57:28<03:28,  5.62s/it]INFO:root: 
Avg Train

100%|███████████████████████████████████████████████████████████████████████████▌| 3980/4000 [3:59:04<01:55,  5.77s/it]INFO:root: 
Avg Training Stats after 3981 global rounds:
03/08 09:09:43 PM |  
Avg Training Stats after 3981 global rounds:
INFO:root:Training Loss : 4.1341413234476934e-05
03/08 09:09:43 PM | Training Loss : 4.1341413234476934e-05
INFO:root:Train Accuracy: 30.27% 

03/08 09:09:43 PM | Train Accuracy: 30.27% 

100%|███████████████████████████████████████████████████████████████████████████▋| 3981/4000 [3:59:10<01:51,  5.88s/it]INFO:root: 
Avg Training Stats after 3982 global rounds:
03/08 09:09:49 PM |  
Avg Training Stats after 3982 global rounds:
INFO:root:Training Loss : 4.032336941236281e-05
03/08 09:09:49 PM | Training Loss : 4.032336941236281e-05
INFO:root:Train Accuracy: 30.27% 

03/08 09:09:49 PM | Train Accuracy: 30.27% 

100%|███████████████████████████████████████████████████████████████████████████▋| 3982/4000 [3:59:17<01:46,  5.93s/it]INFO:root: 
Avg Train

100%|███████████████████████████████████████████████████████████████████████████▉| 3999/4000 [4:01:00<00:06,  6.15s/it]INFO:root: 
Avg Training Stats after 4000 global rounds:
03/08 09:11:38 PM |  
Avg Training Stats after 4000 global rounds:
INFO:root:Training Loss : 3.2902372422540795e-05
03/08 09:11:38 PM | Training Loss : 3.2902372422540795e-05
INFO:root:Train Accuracy: 30.27% 

03/08 09:11:39 PM | Train Accuracy: 30.27% 

100%|████████████████████████████████████████████████████████████████████████████| 4000/4000 [4:01:08<00:00,  3.62s/it]


 Saving checkpoints to ../save/checkpoint\FedCifar100_frac0.02\cifar100_resnet18_T[4000]_C[0.02]_iid[2]_E[1]_B[20]...


INFO:root: 
 Results after 4000 global rounds of training:
03/08 09:11:44 PM |  
 Results after 4000 global rounds of training:
INFO:root:|---- Test Accuracy: 30.27%
03/08 09:11:44 PM | |---- Test Accuracy: 30.27%


## 기본 lr 1

In [5]:
args = easydict.EasyDict({
    "model": 'resnet18',
    'dataset': 'cifar100',
    'gpu': 0,
    'iid': 2,
    'epochs': 4000,
    'optimizer': 'no_weight_decay',
    'seed': 0,
    'norm': 'group_norm',
    'num_users': 500,
    'frac': 0.02,
    'local_ep': 1,
    'local_bs': 20,
    'lr': 1,
    'momentum': 0.9,
    'kernel_num': 9,
    'kernel_sizes': '3,4,5',
    'num_channnels': '1',
    'num_filters': 32,
    'max_pool': 'True',
    'num_classes': 10,
    'unequal': 0,
    'stopping_rounds': 10,
    'verbose': 0,
    'hold_normalize': 0,
    'save_path': '../save/checkpoint',
    'exp_folder': 'FedCifar100_frac0.02',
    'resume': None,
    'lr_decay': 1
})

train_loss, val_acc_list, batch_loss_list, conv_grad_list, fc_grad_list, total_grad_list = main_test(args)
net_loss_list.append(train_loss)
net_acc_list.append(val_acc_list)
net_loss_itr_list.append(batch_loss_list)
net_grad_conv_list.append(conv_grad_list)
net_grad_fc_list.append(fc_grad_list)
net_grad_norm_list.append(total_grad_list)

INFO:root:{'model': 'resnet18', 'dataset': 'cifar100', 'gpu': 0, 'iid': 2, 'epochs': 4000, 'optimizer': 'no_weight_decay', 'seed': 0, 'norm': 'group_norm', 'num_users': 500, 'frac': 0.02, 'local_ep': 1, 'local_bs': 20, 'lr': 0.01, 'momentum': 0.9, 'kernel_num': 9, 'kernel_sizes': '3,4,5', 'num_channnels': '1', 'num_filters': 32, 'max_pool': 'True', 'num_classes': 10, 'unequal': 0, 'stopping_rounds': 10, 'verbose': 0, 'hold_normalize': 0, 'save_path': '../save/checkpoint\\FedCifar100_frac0.02', 'exp_folder': 'FedCifar100_frac0.02', 'resume': None, 'lr_decay': 1}
03/08 09:42:06 PM | {'model': 'resnet18', 'dataset': 'cifar100', 'gpu': 0, 'iid': 2, 'epochs': 4000, 'optimizer': 'no_weight_decay', 'seed': 0, 'norm': 'group_norm', 'num_users': 500, 'frac': 0.02, 'local_ep': 1, 'local_bs': 20, 'lr': 0.01, 'momentum': 0.9, 'kernel_num': 9, 'kernel_sizes': '3,4,5', 'num_channnels': '1', 'num_filters': 32, 'max_pool': 'True', 'num_classes': 10, 'unequal': 0, 'stopping_rounds': 10, 'verbose': 0, '

03/08 09:42:43 PM | ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (relu): ReLU(inplace=True)
      (bn2): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): Gr

num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256


  0%|                                                                                         | 0/4000 [00:00<?, ?it/s]INFO:root: 
Avg Training Stats after 1 global rounds:
03/08 09:42:56 PM |  
Avg Training Stats after 1 global rounds:
03/08 09:42:56 PM |  
Avg Training Stats after 1 global rounds:
INFO:root:Training Loss : 4.13970811367035
03/08 09:42:56 PM | Training Loss : 4.13970811367035
03/08 09:42:56 PM | Training Loss : 4.13970811367035
INFO:root:Train Accuracy: 0.64% 

03/08 09:42:56 PM | Train Accuracy: 0.64% 

03/08 09:42:56 PM | Train Accuracy: 0.64% 

  0%|                                                                               | 1/4000 [00:02<2:39:22,  2.39s/it]INFO:root: 
Avg Training Stats after 2 global rounds:
03/08 09:42:59 PM |  
Avg Training Stats after 2 global rounds:
03/08 09:42:59 PM |  
Avg Training Stats after 2 global rounds:
INFO:root:Training Loss : 4.170010061264039
03/08 09:42:59 PM | Training Loss : 4.170010061264039
03/08 09:42:59 PM | Training 

03/08 09:43:29 PM |  
Avg Training Stats after 15 global rounds:
03/08 09:43:29 PM |  
Avg Training Stats after 15 global rounds:
INFO:root:Training Loss : 3.8630852508544917
03/08 09:43:29 PM | Training Loss : 3.8630852508544917
03/08 09:43:29 PM | Training Loss : 3.8630852508544917
INFO:root:Train Accuracy: 1.85% 

03/08 09:43:29 PM | Train Accuracy: 1.85% 

03/08 09:43:29 PM | Train Accuracy: 1.85% 

  0%|▎                                                                             | 15/4000 [00:35<2:35:32,  2.34s/it]INFO:root: 
Avg Training Stats after 16 global rounds:
03/08 09:43:32 PM |  
Avg Training Stats after 16 global rounds:
03/08 09:43:32 PM |  
Avg Training Stats after 16 global rounds:
INFO:root:Training Loss : 4.030534262657166
03/08 09:43:32 PM | Training Loss : 4.030534262657166
03/08 09:43:32 PM | Training Loss : 4.030534262657166
INFO:root:Train Accuracy: 2.75% 

03/08 09:43:32 PM | Train Accuracy: 2.75% 

03/08 09:43:32 PM | Train Accuracy: 2.75% 

  0%|▎         

...

INFO:root:Train Accuracy: 30.56% 

03/08 11:17:03 PM | Train Accuracy: 30.56% 

03/08 11:17:03 PM | Train Accuracy: 30.56% 

 47%|██████████████████████████████████▌                                       | 1871/4000 [1:34:09<2:15:41,  3.82s/it]INFO:root: 
Avg Training Stats after 1872 global rounds:
03/08 11:17:06 PM |  
Avg Training Stats after 1872 global rounds:
03/08 11:17:06 PM |  
Avg Training Stats after 1872 global rounds:
INFO:root:Training Loss : 0.010897305523976684
03/08 11:17:06 PM | Training Loss : 0.010897305523976684
03/08 11:17:06 PM | Training Loss : 0.010897305523976684
INFO:root:Train Accuracy: 30.69% 

03/08 11:17:07 PM | Train Accuracy: 30.69% 

03/08 11:17:07 PM | Train Accuracy: 30.69% 

 47%|██████████████████████████████████▋                                       | 1872/4000 [1:34:12<2:14:06,  3.78s/it]INFO:root: 
Avg Training Stats after 1873 global rounds:
03/08 11:17:10 PM |  
Avg Training Stats after 1873 global rounds:
03/08 11:17:10 PM |  
Avg Training S

03/08 11:17:55 PM | Training Loss : 0.0175545551860705
03/08 11:17:55 PM | Training Loss : 0.0175545551860705
INFO:root:Train Accuracy: 30.63% 

03/08 11:17:55 PM | Train Accuracy: 30.63% 

03/08 11:17:55 PM | Train Accuracy: 30.63% 

 47%|██████████████████████████████████▊                                       | 1885/4000 [1:35:01<2:08:59,  3.66s/it]INFO:root: 
Avg Training Stats after 1886 global rounds:
03/08 11:17:58 PM |  
Avg Training Stats after 1886 global rounds:
03/08 11:17:58 PM |  
Avg Training Stats after 1886 global rounds:
INFO:root:Training Loss : 0.01904925379436463
03/08 11:17:59 PM | Training Loss : 0.01904925379436463
03/08 11:17:59 PM | Training Loss : 0.01904925379436463
INFO:root:Train Accuracy: 30.69% 

03/08 11:17:59 PM | Train Accuracy: 30.69% 

03/08 11:17:59 PM | Train Accuracy: 30.69% 

 47%|██████████████████████████████████▉                                       | 1886/4000 [1:35:04<2:08:24,  3.64s/it]INFO:root: 
Avg Training Stats after 1887 global roun

03/08 11:18:45 PM |  
Avg Training Stats after 1899 global rounds:
INFO:root:Training Loss : 0.013860598122701049
03/08 11:18:45 PM | Training Loss : 0.013860598122701049
03/08 11:18:45 PM | Training Loss : 0.013860598122701049
INFO:root:Train Accuracy: 30.82% 

03/08 11:18:45 PM | Train Accuracy: 30.82% 

03/08 11:18:45 PM | Train Accuracy: 30.82% 

 47%|███████████████████████████████████▏                                      | 1899/4000 [1:35:51<2:06:19,  3.61s/it]INFO:root: 
Avg Training Stats after 1900 global rounds:
03/08 11:18:49 PM |  
Avg Training Stats after 1900 global rounds:
03/08 11:18:49 PM |  
Avg Training Stats after 1900 global rounds:
INFO:root:Training Loss : 0.0066781794442795205
03/08 11:18:49 PM | Training Loss : 0.0066781794442795205
03/08 11:18:49 PM | Training Loss : 0.0066781794442795205
INFO:root:Train Accuracy: 30.71% 

03/08 11:18:49 PM | Train Accuracy: 30.71% 

03/08 11:18:49 PM | Train Accuracy: 30.71% 

 48%|███████████████████████████████████▏       

 48%|███████████████████████████████████▎                                      | 1912/4000 [1:36:39<2:08:20,  3.69s/it]INFO:root: 
Avg Training Stats after 1913 global rounds:
03/08 11:19:37 PM |  
Avg Training Stats after 1913 global rounds:
03/08 11:19:37 PM |  
Avg Training Stats after 1913 global rounds:
INFO:root:Training Loss : 0.009340413559693842
03/08 11:19:37 PM | Training Loss : 0.009340413559693842
03/08 11:19:37 PM | Training Loss : 0.009340413559693842
INFO:root:Train Accuracy: 30.71% 

03/08 11:19:37 PM | Train Accuracy: 30.71% 

03/08 11:19:37 PM | Train Accuracy: 30.71% 

 48%|███████████████████████████████████▍                                      | 1913/4000 [1:36:42<2:07:23,  3.66s/it]INFO:root: 
Avg Training Stats after 1914 global rounds:
03/08 11:19:40 PM |  
Avg Training Stats after 1914 global rounds:
03/08 11:19:40 PM |  
Avg Training Stats after 1914 global rounds:
INFO:root:Training Loss : 0.00666226637084037
03/08 11:19:40 PM | Training Loss : 0.0066622663

KeyboardInterrupt: 

In [6]:
# test weight decay, num_classes=100
args = easydict.EasyDict({
    "model": 'resnet18',
    'dataset': 'cifar100',
    'gpu': 0,
    'iid': 2,
    'epochs': 4000,
    'optimizer': 'resnet',
    'seed': 0,
    'norm': 'group_norm',
    'num_users': 500,
    'frac': 0.02,
    'local_ep': 1,
    'local_bs': 20,
    'lr': 0.1,
    'momentum': 0.9,
    'kernel_num': 9,
    'kernel_sizes': '3,4,5',
    'num_channnels': '1',
    'num_filters': 32,
    'max_pool': 'True',
    'num_classes': 100,
    'unequal': 0,
    'stopping_rounds': 10,
    'verbose': 0,
    'hold_normalize': 0,
    'save_path': '../save/checkpoint',
    'exp_folder': 'test',
    'resume': None,
    'lr_decay': 1
})

test, test1, test2, test3, test4, test5 = main_test(args)

INFO:root:{'model': 'resnet18', 'dataset': 'cifar100', 'gpu': 0, 'iid': 2, 'epochs': 4000, 'optimizer': 'resnet', 'seed': 0, 'norm': 'group_norm', 'num_users': 500, 'frac': 0.02, 'local_ep': 1, 'local_bs': 20, 'lr': 0.1, 'momentum': 0.9, 'kernel_num': 9, 'kernel_sizes': '3,4,5', 'num_channnels': '1', 'num_filters': 32, 'max_pool': 'True', 'num_classes': 100, 'unequal': 0, 'stopping_rounds': 10, 'verbose': 0, 'hold_normalize': 0, 'save_path': '../save/checkpoint\\test', 'exp_folder': 'test', 'resume': None, 'lr_decay': 1}
03/08 11:20:10 PM | {'model': 'resnet18', 'dataset': 'cifar100', 'gpu': 0, 'iid': 2, 'epochs': 4000, 'optimizer': 'resnet', 'seed': 0, 'norm': 'group_norm', 'num_users': 500, 'frac': 0.02, 'local_ep': 1, 'local_bs': 20, 'lr': 0.1, 'momentum': 0.9, 'kernel_num': 9, 'kernel_sizes': '3,4,5', 'num_channnels': '1', 'num_filters': 32, 'max_pool': 'True', 'num_classes': 100, 'unequal': 0, 'stopping_rounds': 10, 'verbose': 0, 'hold_normalize': 0, 'save_path': '../save/checkpoi

num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:32
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:64
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128
num_channels_per_group:128


INFO:root:ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (relu): ReLU(inplace=True)
      (bn2): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): GroupNorm2d(

num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256
num_channels_per_group:256


03/08 11:27:34 PM | ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (relu): ReLU(inplace=True)
      (bn2): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): Gr

03/08 11:27:34 PM | ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (relu): ReLU(inplace=True)
      (bn2): GroupNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): Gr

03/08 11:28:39 PM | Train Accuracy: 0.65% 

03/08 11:28:39 PM | Train Accuracy: 0.65% 

03/08 11:28:39 PM | Train Accuracy: 0.65% 

  0%|                                                                               | 5/4000 [00:20<4:28:59,  4.04s/it]INFO:root: 
Avg Training Stats after 6 global rounds:
03/08 11:28:42 PM |  
Avg Training Stats after 6 global rounds:
03/08 11:28:42 PM |  
Avg Training Stats after 6 global rounds:
03/08 11:28:42 PM |  
Avg Training Stats after 6 global rounds:
INFO:root:Training Loss : 4.303892445564269
03/08 11:28:42 PM | Training Loss : 4.303892445564269
03/08 11:28:42 PM | Training Loss : 4.303892445564269
03/08 11:28:42 PM | Training Loss : 4.303892445564269
INFO:root:Train Accuracy: 1.34% 

03/08 11:28:43 PM | Train Accuracy: 1.34% 

03/08 11:28:43 PM | Train Accuracy: 1.34% 

03/08 11:28:43 PM | Train Accuracy: 1.34% 

  0%|                                                                               | 6/4000 [00:24<4:29:13,  4.04s/it]INFO:root: 


KeyboardInterrupt: 

## ws+clip lr 1

In [ ]:
args = easydict.EasyDict({
    "model": 'resnet18_ws',
    'dataset': 'cifar100',
    'gpu': 0,
    'iid': 2,
    'epochs': 4000,
    'optimizer': 'clip',
    'seed': 0,
    'norm': 'group_norm',
    'num_users': 500,
    'frac': 0.02,
    'local_ep': 1,
    'local_bs': 20,
    'lr': 1,
    'momentum': 0.9,
    'kernel_num': 9,
    'kernel_sizes': '3,4,5',
    'num_channnels': '1',
    'num_filters': 32,
    'max_pool': 'True',
    'num_classes': 10,
    'unequal': 0,
    'stopping_rounds': 10,
    'verbose': 0,
    'hold_normalize': 0,
    'save_path': '../save/checkpoint',
    'exp_folder': 'FedCifar100_frac0.02',
    'resume': None,
    'lr_decay': 0.9
})

train_loss, val_acc_list, batch_loss_list, conv_grad_list, fc_grad_list, total_grad_list = main_test(args)
net_loss_list.append(train_loss)
net_acc_list.append(val_acc_list)
net_loss_itr_list.append(batch_loss_list)
net_grad_conv_list.append(conv_grad_list)
net_grad_fc_list.append(fc_grad_list)
net_grad_norm_list.append(total_grad_list)

## ws clip lr 5

In [ ]:
args = easydict.EasyDict({
    "model": 'resnet18_ws',
    'dataset': 'cifar100',
    'gpu': 0,
    'iid': 2,
    'epochs': 4000,
    'optimizer': 'clip',
    'seed': 0,
    'norm': 'group_norm',
    'num_users': 500,
    'frac': 0.02,
    'local_ep': 1,
    'local_bs': 20,
    'lr': 5,
    'momentum': 0.9,
    'kernel_num': 9,
    'kernel_sizes': '3,4,5',
    'num_channnels': '1',
    'num_filters': 32,
    'max_pool': 'True',
    'num_classes': 10,
    'unequal': 0,
    'stopping_rounds': 10,
    'verbose': 0,
    'hold_normalize': 0,
    'save_path': '../save/checkpoint',
    'exp_folder': 'FedCifar100_frac0.02',
    'resume': None,
    'lr_decay': 0.9
})

train_loss, val_acc_list, batch_loss_list, conv_grad_list, fc_grad_list, total_grad_list = main_test(args)
net_loss_list.append(train_loss)
net_acc_list.append(val_acc_list)
net_loss_itr_list.append(batch_loss_list)
net_grad_conv_list.append(conv_grad_list)
net_grad_fc_list.append(fc_grad_list)
net_grad_norm_list.append(total_grad_list)

## ws+group lr 0.1
->GN+ws효과 확인, lr0.01이 더 best인 경우 교체

In [ ]:
args = easydict.EasyDict({
    "model": 'resnet18_ws',
    'dataset': 'cifar100',
    'gpu': 0,
    'iid': 2,
    'epochs': 4000,
    'optimizer': 'no_weight_decay',
    'seed': 0,
    'norm': 'group_norm',
    'num_users': 500,
    'frac': 0.02,
    'local_ep': 1,
    'local_bs': 20,
    'lr': 0.1,
    'momentum': 0.9,
    'kernel_num': 9,
    'kernel_sizes': '3,4,5',
    'num_channnels': '1',
    'num_filters': 32,
    'max_pool': 'True',
    'num_classes': 10,
    'unequal': 0,
    'stopping_rounds': 10,
    'verbose': 0,
    'hold_normalize': 0,
    'save_path': '../save/checkpoint',
    'exp_folder': 'FedCifar100_frac0.02## 기본 lr 0.1',
    'resume': None,
    'lr_decay': 1
})

train_loss, val_acc_list, batch_loss_list, conv_grad_list, fc_grad_list, total_grad_list = main_test(args)
net_loss_list.append(train_loss)
net_acc_list.append(val_acc_list)
net_loss_itr_list.append(batch_loss_list)
net_grad_conv_list.append(conv_grad_list)
net_grad_fc_list.append(fc_grad_list)
net_grad_norm_list.append(total_grad_list)

## ws+gn 은 large learning rate가능? lr1

In [ ]:
args = easydict.EasyDict({
    "model": 'resnet18_ws',
    'dataset': 'cifar100',
    'gpu': 0,
    'iid': 2,
    'epochs': 4000,
    'optimizer': 'no_weight_decay',
    'seed': 0,
    'norm': 'group_norm',
    'num_users': 500,
    'frac': 0.02,
    'local_ep': 1,
    'local_bs': 20,
    'lr': 1,
    'momentum': 0.9,
    'kernel_num': 9,
    'kernel_sizes': '3,4,5',
    'num_channnels': '1',
    'num_filters': 32,
    'max_pool': 'True',
    'num_classes': 10,
    'unequal': 0,
    'stopping_rounds': 10,
    'verbose': 0,
    'hold_normalize': 0,
    'save_path': '../save/checkpoint',
    'exp_folder': 'FedCifar100_frac0.02## 기본 lr 0.1',
    'resume': None,
    'lr_decay': 1
})

train_loss, val_acc_list, batch_loss_list, conv_grad_list, fc_grad_list, total_grad_list = main_test(args)
net_loss_list.append(train_loss)
net_acc_list.append(val_acc_list)
net_loss_itr_list.append(batch_loss_list)
net_grad_conv_list.append(conv_grad_list)
net_grad_fc_list.append(fc_grad_list)
net_grad_norm_list.append(total_grad_list)

In [ ]:
# 추가 실험. GN+CLip, Ws+clip(not gn) -> large learning rate // 그리고 gn lr0.001 or lr0.001